## Input

In [16]:
import pandas as pd
df = pd.read_csv(r'C:\Users\Morga\programsMG\TextMining\Tweets.csv')

In [17]:
# Dropping the row 
df.drop([314], inplace = True)

## Preprocessing

In [18]:
#seperating the label
y=df['sentiment'].values
M=df['text'].tolist()

In [19]:
import re
X1=[]
for item in M:
    X1.append(re.sub(r'([a-zA-Z0-9_<>-])\1+', r'\1\1', item))

In [20]:
tempX=[]
for item in X1:
    iteml=item.lower()
    item1=iteml.replace('2day ','today ')
    item2=item1.replace('2moro ','tomorrow ')
    item3=item2.replace('2morrow ','tomorrow ')
    item4=item3.replace('2night ','tonight ')
    item5=item4.replace('2nite ','tonight ')
    item6=item5.replace('b-day ','birthday ')
    item7=item6.replace('b4 ','before ')
    item8=item7.replace('bb ','be back ')
    item9=item8.replace('bbl ','be back later ')
    item10=item9.replace('bc ','because ')
    item11=item10.replace('bday ','birthday ')
    item12=item11.replace('belive ','believe ')
    item13=item12.replace('bf ','boyfriend ')
    item14=item13.replace('bff ','best freind forever ')
    item15=item14.replace('brb ','be right back ')
    item16=item15.replace('bros ','bro ')
    item17=item16.replace('bs ','bullshit ')
    item18=item17.replace('btw ','by the way ')
    item19=item18.replace('dat ','that ')
    item20=item19.replace('doc ','doctor ')
    item21=item20.replace('docs ','doctor ')
    item22=item21.replace('hott ','hot ')
    item23=item22.replace('fb ','facebook ')
    item24=item23.replace('jk ','just kidding ')
    item25=item24.replace('jst ','just ')
    item26=item25.replace(' ng ',' nice game ')
    item27=item26.replace('nt ','nice try ')
    item28=item27.replace('ok ','okay ')
    item29=item28.replace('okayy ','okay ')
    item30=item29.replace('omgg ','omg ')
    item31=item30.replace('ppl ','people ')
    item32=item31.replace('tonite ','tonight ')
    item33=item32.replace(' u ',' you ')
    item34=item33.replace('u2 ','you too ')
    item35=item34.replace('ugg ','ugh ')
    item36=item35.replace('uggh ','ugh ')
    item37=item36.replace('uh ','ugh ')
    item38=item37.replace('uhh ','ugh ')
    item39=item38.replace('umm ','um ')
    item40=item39.replace('ur ','your ')
    item41=item40.replace('waah ','waa ')
    item42=item41.replace('wah ','waa ')
    item43=item42.replace('waay ','waa ')
    item44=item43.replace('xoxo ','xo ')
    item45=item44.replace('xx ','xo ')
    item46=item45.replace(' y ',' why ')
    item47=item46.replace('whyy ','why ')
    item48=item47.replace('yaay ','ya ')
    item49=item48.replace('yah ','ya ')
    item50=item49.replace('juss ','just ')
    item51=item50.replace('whassqoodd ','whats good') 
    item52=item51.replace('soo ','so ')
    tempX.append(item52)

In [21]:
# import TweetTokenizer() method from nltk
from nltk.tokenize import TweetTokenizer
tk = TweetTokenizer()

In [22]:
# create a function for the tweet tokenizer from NLTK
def tok(text):
    tt = TweetTokenizer()
    return tt.tokenize(text)

In [23]:
#selecting the stop words
stopwords =['!','#','$','%','&',"'",'*','(',')','+', ',','-','.','/',':',';','<','=','>'
              ,'?','@','[','\\',']','^','_','`','{','|','}','~','¿','#bgt','#fail','#fb'
              ,'#ff','#followfriday','#hhrs','#judday','#sanctuarysunday','#starwarsday'
              ,'#twpp','>','_','_127','__','_b','_benson','_c','_carter','_d','_guy','_henrie'
              ,'_j','_m','_marie','_skies','_x','_xo']

In [24]:
# creating a function for the pipeline to remove stop words
def remove_stop(tokens):
    return [t for t in tokens if t not in stopwords]

In [25]:
# creating a function to use each function from the pipeline on the texts
def prepare(text, pipeline):
    tokens = text
    for transform in pipeline:
        tokens = transform(tokens)
    return tokens     

In [26]:
# selecting the functions for the pipeline
pipeline = [tok,remove_stop,]
# using the pipelined list of functions.
Xtemp=[]
for tweet in tempX:
    Xtemp.append(prepare(tweet,pipeline))

In [27]:
#creating a new list
X=[]
#joining the lists of lists
#so a list of stirngs remains
for tweet in Xtemp:
    X.append(' '.join(tweet))

## Vectorization

In [28]:
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
stemmer = PorterStemmer()
analyzer = TfidfVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

In [29]:
# import the LinearSVC module
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
unigram_tfidf_vectorizer_Tweet_toke_no_stop_stem = TfidfVectorizer(encoding='latin-1'
                                                                   , use_idf=True
                                                                   , min_df=5
                                                                   ,tokenizer=tok
                                                                   ,analyzer=stemmed_words)

In [30]:
vecs = unigram_tfidf_vectorizer_Tweet_toke_no_stop_stem.fit_transform(X)

vecsdf=pd.DataFrame(vecs.toarray(),
            columns=unigram_tfidf_vectorizer_Tweet_toke_no_stop_stem.get_feature_names_out())

## Adding Feature Sets

In [31]:
words=[]
for review in X:
    words.append(tok(review))

In [32]:
word_count =[]
for review in words:
    count =0
    for word in review:
        count=count+1
    word_count.append(count)

In [33]:
import numpy as np
word_array = np.asarray(word_count)
word_norm = (word_array - word_array.min())/ (word_array.max() - word_array.min())

In [34]:
vecsdf['word_norm']=word_norm

In [35]:
import re
def has_negation(post):
    pattern_neg_1 = re.compile(r'\b(not|no|never)\b')
    pattern_neg_2 = re.compile(r'\b([a-z]+less)\b')
    if pattern_neg_1.search(post.lower()) or pattern_neg_2.search(post.lower()):
        return 1
    else: 
        return 0

In [36]:
neg_count=[]
for item in X:
    neg_count.append(has_negation(item))

In [37]:
neg_array = np.asarray(neg_count)
neg_norm = (neg_array - neg_array.min())/ (neg_array.max() - neg_array.min())

In [38]:
vecsdf['neg_norm']=neg_norm

## Preforming Gridsearchcv

In [44]:
from sklearn.svm import SVC
#initialize the model
svc = SVC()

In [45]:
#from sklearn.model_selection import GridSearchCV
#param_grid = {'C': [0.1, 1, 10, 100],'kernel': ['linear']}  
#grid_svm_ngram = GridSearchCV(svc, param_grid, refit = True, verbose = 3, cv=3) 
#grid_svm_ngram.fit(vecsdf, y)
#grid_svm_ngram.cv_results_

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV 1/3] END ..............C=0.1, kernel=linear;, score=0.630 total time=18.9min
[CV 2/3] END ..............C=0.1, kernel=linear;, score=0.629 total time=18.0min
[CV 3/3] END ..............C=0.1, kernel=linear;, score=0.634 total time=18.6min
[CV 1/3] END ................C=1, kernel=linear;, score=0.708 total time=14.4min
[CV 2/3] END ................C=1, kernel=linear;, score=0.712 total time=14.3min
[CV 3/3] END ................C=1, kernel=linear;, score=0.713 total time=14.4min
[CV 1/3] END ...............C=10, kernel=linear;, score=0.678 total time=16.4min
[CV 2/3] END ...............C=10, kernel=linear;, score=0.680 total time=16.5min
[CV 3/3] END ...............C=10, kernel=linear;, score=0.674 total time=16.5min
[CV 1/3] END ..............C=100, kernel=linear;, score=0.646 total time=36.0min
[CV 2/3] END ..............C=100, kernel=linear;, score=0.651 total time=36.3min
[CV 3/3] END ..............C=100, kernel=linear;,

{'mean_fit_time': array([ 797.43565663,  614.29128806,  776.90750162, 1965.3668735 ]),
 'std_fit_time': array([20.28127319,  2.2663014 ,  4.06782015, 17.79826103]),
 'mean_score_time': array([312.66564234, 249.85184956, 211.75202394, 193.0925192 ]),
 'std_score_time': array([8.80795456, 0.66921283, 0.15327728, 0.23836649]),
 'param_C': masked_array(data=[0.1, 1, 10, 100],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['linear', 'linear', 'linear', 'linear'],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 0.1, 'kernel': 'linear'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 100, 'kernel': 'linear'}],
 'split0_test_score': array([0.63045852, 0.70840611, 0.67772926, 0.64606987]),
 'split1_test_score': array([0.62925764, 0.71200873, 0.68002183, 0.65131004]),
 'split2_test_score': array([0.63438865, 0.7

In [46]:
#svm_ngram_df=pd.DataFrame(grid_svm_ngram.cv_results_)

In [47]:
#svm_ngram_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,797.435657,20.281273,312.665642,8.807955,0.1,linear,"{'C': 0.1, 'kernel': 'linear'}",0.630459,0.629258,0.634389,0.631368,0.002191,4
1,614.291288,2.266301,249.851850,0.669213,1,linear,"{'C': 1, 'kernel': 'linear'}",0.708406,0.712009,0.713319,0.711245,0.002077,1
2,776.907502,4.067820,211.752024,0.153277,10,linear,"{'C': 10, 'kernel': 'linear'}",0.677729,0.680022,0.673581,0.677111,0.002666,2
3,1965.366874,17.798261,193.092519,0.238366,100,linear,"{'C': 100, 'kernel': 'linear'}",0.646070,0.651310,0.645524,0.647635,0.002608,3
